In [ ]:
from pathlib import Path
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use("seaborn-v0_8-darkgrid")

# Localisation du fichier CSV
current_dir = Path().resolve()
project_root = current_dir.parent
data_path = project_root / "Data" / "Dataset of weighing station temperature measurements.csv"

print("Fichier CSV :", data_path)

df = pd.read_csv(data_path, sep=";")

# Aperçu
display(df.head())
print(df.columns)

In [ ]:
# convertir la colonne Time en datetime et la mettre en index
df['Time'] = pd.to_datetime(df['Time'])
df = df.set_index('Time').sort_index()

df.head()
print(df)

In [ ]:
# Colonnes de température à l'intérieur du puits
low_cols = [c for c in df.columns if "Low" in c]
mid_cols = [c for c in df.columns if "Mid" in c]
top_cols = [c for c in df.columns if "Top" in c]

# trie par numéro de capteur S1, S2, ... 
def sort_by_sensor(col_list):
    return sorted(col_list, key=lambda x: int(x.split("-S")[1]))

low_cols = sort_by_sensor(low_cols)
mid_cols = sort_by_sensor(mid_cols)
top_cols = sort_by_sensor(top_cols)

# P6 : données valides seulement à partir du 26 janvier 2024 
date_debut_P6 = pd.to_datetime("2024-01-26")

p6_sensors = [24, 25, 26, 27, 28, 29]

p6_low_cols = [low_cols[s-1] for s in p6_sensors]
p6_mid_cols = [mid_cols[s-1] for s in p6_sensors]
p6_top_cols = [top_cols[s-1] for s in p6_sensors]

p6_all_cols = p6_low_cols + p6_mid_cols + p6_top_cols

# met à NaN toutes les valeurs de P6 avant le 26 janvier
df.loc[df.index < date_debut_P6, p6_all_cols] = np.nan

# Définir les zones une seule fois
zones = {
    "P1": [1, 2, 3, 4, 5],
    "P2": [6, 7, 8, 9],
    "P3": [10, 11, 12, 13],
    "P4": [14, 15, 16, 17, 18],
    "P5": [19, 20, 21, 22, 23],
    "P6": [24, 25, 26, 27, 28, 29],
}

# Seuil et paramètres globaux
on_threshold = 3.5
incertitude_max = 2.0
bins = 60

print("Nb capteurs LOW :", len(low_cols))
print("Nb capteurs MID :", len(mid_cols))
print("Nb capteurs TOP :", len(top_cols))

print("\nExemples LOW :", low_cols[:5])
print("Exemples MID :", mid_cols[:5])
print("Exemples TOP :", top_cols[:5])

# --- Centralisation : pré-calculer les séries utiles pour chaque zone ---
# zones_data[zone_name] = {
#     'T_low': Series, 'T_mid': Series, 'T_top': Series,
#     'dT': Series, 'dT_smooth': Series, 'heater_on': Series,
#     'T_int': Series
# }
zones_data = {}
for zone_name, sondes in zones.items():
    T_low_zone = df[[low_cols[s-1] for s in sondes]].mean(axis=1)
    T_mid_zone = df[[mid_cols[s-1] for s in sondes]].mean(axis=1)
    T_top_zone = df[[top_cols[s-1] for s in sondes]].mean(axis=1)

    dT_zone = T_top_zone - T_low_zone
    dT_smooth_zone = dT_zone.rolling(5, center=True, min_periods=1).mean()
    heater_on_zone = dT_smooth_zone > on_threshold
    T_int_zone = (T_low_zone + T_mid_zone + T_top_zone) / 3

    zones_data[zone_name] = {
        'T_low': T_low_zone,
        'T_mid': T_mid_zone,
        'T_top': T_top_zone,
        'dT': dT_zone,
        'dT_smooth': dT_smooth_zone,
        'heater_on': heater_on_zone,
        'T_int': T_int_zone,
        'sondes': sondes
    }

In [ ]:
# Histogrammes de ΔT local dans différentes zones du puits

for zone_name in zones.keys():
    dT_zone = zones_data[zone_name]['dT']

    plt.figure(figsize=(8,5))
    plt.hist(dT_zone.dropna(), bins=bins, edgecolor="black", alpha=0.7)
    plt.axvline(incertitude_max, color="red", linestyle="--", label="Incertitude max")
    plt.xlabel("ΔT = T_top - T_low [°C]")
    plt.ylabel("Fréquence")
    plt.title(f"Histogramme de ΔT local – {zone_name} (capteurs {zones_data[zone_name]['sondes']})")
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.show()

In [ ]:
# Visualisation de ΔT local lissé et détection ON/OFF pour une zone choisie
# Choisissez la zone à tracer. Par défaut : P6 si présente, sinon la dernière définie.
zone_to_plot = 'P6' if 'P6' in zones else list(zones.keys())[-1]
zone_data = zones_data[zone_to_plot]
sondes = zone_data['sondes']

dT_smooth_zone = zone_data['dT_smooth']
heater_on = zone_data['heater_on']

# Tracé
plt.figure(figsize=(14,5))
plt.plot(dT_smooth_zone, label="ΔT (TOP-LOW) lissé", color="tab:red")
plt.axhline(on_threshold, color="black", linestyle="--", label=f"Seuil ON = {on_threshold} °C")

plt.fill_between(dT_smooth_zone.index, on_threshold, dT_smooth_zone,
                 where=heater_on, color="tab:red", alpha=0.2, label="Aérotherme ON")

plt.xlabel("Temps")
plt.ylabel("ΔT local [°C]")
plt.title(f"Zone {zone_to_plot} (sondes {sondes[0]}–{sondes[-1]}) | ΔT local et détection ON/OFF")
plt.legend()
plt.grid(True, alpha=0.8)

# Format et fréquence des dates sur l'axe X (mdates importé dans la cellule 1)
plt.gca().xaxis.set_major_locator(mdates.WeekdayLocator(interval=1))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gcf().autofmt_xdate()

plt.show()

In [ ]:
# Pourcentage du temps ON en fonction de T_ext et autres infos (zone choisie)

# Utiliser la zone tracée précédemment
zone_to_use = zone_to_plot
zone_data = zones_data[zone_to_use]
heater_on_zone = zone_data['heater_on']
T_int_zone = zone_data['T_int']

T_ext = df["Outdoor temperature [deg. C]"]

tmp = pd.DataFrame({
    "T_ext_arrondie": T_ext.round().astype("Int64"),
    "ON": heater_on_zone.astype(int)
})
pourcentage_on_ext = tmp.groupby("T_ext_arrondie")["ON"].mean().sort_index() * 100

print(pourcentage_on_ext)

T_ext_ON  = T_ext[heater_on_zone]
T_ext_OFF = T_ext[~heater_on_zone]

T_int_ON  = T_int_zone[heater_on_zone]
T_int_OFF = T_int_zone[~heater_on_zone]

print(f"T_ext moyenne quand heater ON  : {T_ext_ON.mean():.2f} °C")
print(f"T_ext moyenne quand heater OFF : {T_ext_OFF.mean():.2f} °C")
print(f"T_int locale moyenne quand ON  : {T_int_ON.mean():.2f} °C")
print(f"T_int locale moyenne quand OFF : {T_int_OFF.mean():.2f} °C")

plt.figure(figsize=(7,5))
plt.plot(pourcentage_on_ext.index, pourcentage_on_ext.values, marker="o")
plt.xlabel("Température extérieure arrondie [°C]")
plt.ylabel("Pourcentage du temps ON [%]")
plt.title(f"Pourcentage du temps ON en fonction de T_ext (sondes S{sondes[0]}–S{sondes[-1]})")
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:


T_ext_all = df["Outdoor temperature [deg. C]"]

list_zones_df = []

for zone_name, sondes in zones.items():
    # Températures LOW / MID / TOP pour cette zone
    T_low_zone = df[[low_cols[s-1] for s in sondes]].mean(axis=1)
    T_mid_zone = df[[mid_cols[s-1] for s in sondes]].mean(axis=1)
    T_top_zone = df[[top_cols[s-1] for s in sondes]].mean(axis=1)

    # ΔT local + lissage
    dT_local_zone = T_top_zone - T_low_zone
    dT_smooth_zone = dT_local_zone.rolling(5, center=True, min_periods=1).mean()

    # Détection ON/OFF
    heater_on_zone = (dT_smooth_zone > on_threshold)

    # Température intérieure moyenne dans la zone
    T_int_zone = (T_low_zone + T_mid_zone + T_top_zone) / 3

    # Petit DataFrame final de la zone
    df_zone_final = pd.DataFrame({
        "zone": zone_name,
        "T_ext": T_ext_all,
        "T_ext_arrondie": T_ext_all.round().astype("Int64"),
        "ON": heater_on_zone.astype(int),
        "T_int": T_int_zone,
    })

    list_zones_df.append(df_zone_final)



df_all_zones = pd.concat(list_zones_df, ignore_index=True)
print(df_all_zones)



T_ext_moy = T_ext_all.mean()

# Moyenne globale du puits (toutes sondes LOW/MID/TOP, tout le temps)
T_puits_moy = df[low_cols + mid_cols + top_cols].mean(axis=1).mean()

print(f"T_ext moyenne : {T_ext_moy:.2f} °C")
print(f"T_puits moyenne : {T_puits_moy:.2f} °C")


In [ ]:
# Pourcentage du temps ON en fonction de T_ext (global, toutes zones confondues)
pourcentage_on_ext_global = (
    df_all_zones
        .groupby("T_ext_arrondie")["ON"]
        .mean()
        .sort_index() * 100
)

print(pourcentage_on_ext_global)


In [ ]:
# Températures moyennes globales quand heater ON/OFF
T_ext_global_ON  = df_all_zones.loc[df_all_zones["ON"] == 1, "T_ext"].mean()
T_ext_global_OFF = df_all_zones.loc[df_all_zones["ON"] == 0, "T_ext"].mean()

T_int_global_ON  = df_all_zones.loc[df_all_zones["ON"] == 1, "T_int"].mean()
T_int_global_OFF = df_all_zones.loc[df_all_zones["ON"] == 0, "T_int"].mean()

print(f"T_ext moyenne globale quand heater ON  : {T_ext_global_ON:.2f} °C")
print(f"T_ext moyenne globale quand heater OFF : {T_ext_global_OFF:.2f} °C")
print(f"T_int locale moyenne globale ON        : {T_int_global_ON:.2f} °C")
print(f"T_int locale moyenne globale OFF       : {T_int_global_OFF:.2f} °C")


In [ ]:
plt.figure(figsize=(7,5))
plt.plot(
    pourcentage_on_ext_global.index,
    pourcentage_on_ext_global.values,
    marker="o"
)
plt.xlabel("Température extérieure arrondie [°C]")
plt.ylabel("Pourcentage du temps ON [%]")
plt.title("Pourcentage du temps ON en fonction de T_ext (moyenne des 5 zones)")
plt.grid(True, alpha=0.3)
plt.show()


Les histogrammes de ΔT = T_top − T_low montrent que, dans toutes les zones étudiées, la majorité des valeurs sont largement au-dessus de l’incertitude des capteurs (~2 °C) : on observe typiquement des ΔT de 4 à 7 °C lorsque l’aérotherme fonctionne, ce qui confirme une stratification locale importante entre le bas et le haut du puits. Dans la zone droite, la distribution bimodale (un pic près de 0–1 °C et un autre vers 5–6 °C) distingue clairement les périodes sans chauffage des périodes où l’aérotherme est actif.

En utilisant un seuil de ΔT lissé de 3,5 °C pour détecter l’état ON/OFF, on montre que l’aérotherme est presque toujours ON quand la température extérieure est très froide (en dessous d’environ −15 à −20 °C), puis son temps de fonctionnement diminue rapidement à l’approche de 0 °C et devient quasi nul au-delà de 5 °C. Globalement, sur la période étudiée, T_ext moyenne est d’environ −6 °C alors que T_int moyenne est autour de 21–22 °C. Quand l’aérotherme est ON, l’extérieur est en moyenne plus froid (≈ −7,7 °C) et l’intérieur atteint ~25 °C; lorsqu’il est OFF, l’extérieur est plus doux (≈ −4 °C) et l’intérieur descend autour de 17–18 °C. En résumé, l’aérotherme est fortement sollicité par temps froid, crée des ΔT verticaux de plusieurs degrés et permet de maintenir des températures intérieures nettement supérieures à la température extérieure.
